# At-Risk Diabetes Classifier

**Author:** Joseph Curtis

# Overview

This notebook presents the At-Risk Diabetes Classifier, a machine learning project aimed at identifying individuals at risk of diabetes based on a variety of health indicators. By analyzing a dataset with features such as BMI, age, smoking status, physical activity, and more, we develop and train a supervised model to predict the likelihood of diabetes.

The project demonstrates:
- Data preprocessing and cleaning techniques.
- Exploratory data analysis (EDA) to understand the relationships within the data.
- Feature selection and engineering to prepare the data for modeling.
- Evaluation and comparison of different machine learning models, including Logistic Regression and Random Forest, to find the most effective predictor of diabetes risk.

This work is part of an ongoing exploration into how data science can be applied to healthcare data to predict disease risk and support early intervention strategies.

For more information, source code, and related resources, please visit the project's GitHub repository at [github.com/joseph-curtis](https://github.com/joseph-curtis/data-science-diabetes-classifier)

- <a href="https://github.com/joseph-curtis/data-science-diabetes-classifier" target="_blank"><img align="left" alt="Github" title="Open in Github" width="144" height="42" src="https://img.shields.io/badge/GitHub-372a4a?style=for-the-badge&logo=github&logoColor=white" /></a>

# Google Colab

Run the following cell when in the Google Colab environment in order to import the data sources from Kaggle.

<p><a target="_blank" href="https://colab.research.google.com/github/joseph-curtis/data-science-diabetes-classifier/blob/master/colab-notebook.ipynb">
  <img align="left" alt="Google Colab" title="Open In Colab" width="167" height="30" src="https://colab.research.google.com/assets/colab-badge.svg" />
</a></p>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'diabetes-health-indicators-dataset:\
https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1703281%2F2789260%2F\
bundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential\
%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240318%252F\
auto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240318T061338Z%26X-Goog-\
Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07bdb86ecd7\
0c483126055dcba8314541ec88e567b9ae028bbbde9b68d274d5ec1c6e3d0c1859178b6bd18f964\
9b8f5dd99e03049a0d45c1397463451d0d36f83e756b0776028f46a89fe8b0c511bc7b73b0faa2f\
2934c0e7dfa27049055525960d1b572a89d5ac7319c6aaa61caa5408b85dbb928ec19625e6a11cd\
a1d6d4d70862a65d5397d43b247886928e2ad4f8d99fde0953855b6082face35e28682484ffffbd\
3151026ae65a9f8217e86f983d1ac299e52d6d21d1fdbef7f03e356ee59c6645f8e310e506c3a63\
4a038bb71a59d80e81b2c77f9d150ceaa5cb954028a30b9b5cc35c0e8f59801c23953f2293e1e44\
3648b1cb2ef6fcc6e72b380a32a,d/julnazz/diabetes-health-indicators-dataset:\
https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4066291%2F7062872%2F\
bundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential\
%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240318%252F\
auto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240318T061338Z%26X-Goog-\
Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72918207ec3\
1108b2099a2604d603e2313e617b29a0b66dcf71b93f1f5e586210ab1637c8b08b43a6ac98abaa2\
cf2310f9db52880746ad9ec4c3a6fd7a6512577f51feb3922ede22b63845e82c8c373c9cce0abdd\
cf6efb55dc855344af8996939b4cc6f2740bc0e4942d81bb0ca92415cce88873ee1cfa4631687ba\
e6dbcb680cdd941f21e62170b1edd284afb7fb9855111f1a19f5419e3af2b521304ef0ed5326fe9\
685eb4e98751fe0622c12d249ccf52673e3711b5edcb0dd485e04f9db5317d0758f354e4ace0381\
c1c997e657a76ad058cb03f67565a50c7c0ff6dd43a1a8aaad88948ada23635548ccf96389bc632\
666c84a8a4ad9bffb0906f630f6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 6324278 bytes downloaded
Downloaded and uncompressed: diabetes-health-indicators-dataset
[==================================================] 5555220 bytes downloaded
Downloaded and uncompressed: d/julnazz/diabetes-health-indicators-dataset
Data source import complete.


# Data Preparation and Initial Analysis

Here we focus on preparing and analyzing health indicator data for diabetes from the years 2015 and 2021. The goal is to combine these datasets to form a comprehensive view of the data, which can later be used for machine learning modeling. The following steps are taken:

1. Import Libraries
2. Load the Data
3. Combine the DataFrames
4. Rename the column 'Diabetes_binary' for brevity

In [ ]:
# import Libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn import linear_model, metrics, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler

# file names and urls
filepath_2015 = "/kaggle/input/diabetes-health-indicators-dataset\
/diabetes_binary_5050split_health_indicators_BRFSS2015.csv"
filepath_2021 = "/kaggle/input/d/julnazz/diabetes-health-indicators-dataset\
/diabetes_binary_5050split_health_indicators_BRFSS2021.csv"
df1 = pd.read_csv(filepath_2015)
df2 = pd.read_csv(filepath_2021)

# Combine the two DataFrames
combined_df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

# Rename the 'Diabetes_binary' column to 'Diabetes'
combined_df.rename(columns={"Diabetes_binary": "Diabetes"}, inplace=True)

# Display the first few rows of the combined dataframe
# and its shape to verify the combination
combined_df_info = combined_df.head(), combined_df.shape

combined_df_info  # output

# Removing Features

Here we remove features deemed irrelevant for modeling purposes. We specify a list of columns to be removed and then drop these columns from the combined DataFrame.

We reverse the values of the "General Health" feature to make it more clear to the audience when graphed. This makes a value of 5 = "excellent" and value of 1 = "poor".

In [ ]:
# Remove irrelevant features from the combined dataset
columns_to_remove = ["CholCheck", "AnyHealthcare", "NoDocbcCost", "Education", "Income"]
reduced_df = combined_df.drop(columns=columns_to_remove)

# Reverse the values of 'GenHlth'
reduced_df["GenHlth"] = 6 - reduced_df["GenHlth"]

# Display the first few rows of the reduced dataframe to verify the removal
reduced_df.head()

# Check for Missing Values

Here we check for missing values in the combined dataset. Note there are *NO* missing values found.

In [ ]:
# Check for missing values in the reduced dataset
missing_values = reduced_df.isnull().sum()

missing_values  ## no missing values found

# Data Type Optimization Analysis

Here we perform a preliminary analysis aimed at optimizing data storage and processing efficiency by examining the range of values for selected features within the reduced dataset. Here we see that all numerical values are well within the range of an 8-bit integer (this dataset has no numbers with decimal values).

We also calculate the memory used before we adjust the data types-- this is our baseline for memory usage.

In [ ]:
# Check range of values of specified features to determine suitable data types
features_to_optimize = ["BMI", "GenHlth", "MentHlth", "PhysHlth", "Age"]
data_types_optimization = (
    reduced_df[features_to_optimize].describe().loc[["min", "max"]]
)

# Memory used before reducing data types
memory_before = reduced_df.memory_usage(index=True).sum()

data_types_optimization

# Data Type Conversion for Efficiency

We enhance the dataset's memory efficiency by converting specified binary columns to boolean data types. We print the memory usage before and after the operation to demonstrate the effectiveness of this optimization in reducing the dataset's memory consumption.

In [ ]:
# scale data types down to reduce memory footprint
reduced_df["BMI"] = reduced_df["BMI"].astype("float32")
reduced_df["GenHlth"] = reduced_df["GenHlth"].astype("int8")
reduced_df["MentHlth"] = reduced_df["MentHlth"].astype("int8")
reduced_df["PhysHlth"] = reduced_df["PhysHlth"].astype("int8")
reduced_df["Age"] = reduced_df["Age"].astype("int8")

# convert 1/0 binary columns to boolean values
binary_columns = [
    "Diabetes",
    "HighBP",
    "HighChol",
    "Smoker",
    "Stroke",
    "HeartDiseaseorAttack",
    "PhysActivity",
    "Fruits",
    "Veggies",
    "HvyAlcoholConsump",
    "DiffWalk",
    "Sex",
]
for column in binary_columns:
    reduced_df[column] = reduced_df[column].astype("bool")

# memory size after data type reduction
memory_after = reduced_df.memory_usage(index=True).sum()

print("Dataframe memory used before:", memory_before)
print("Dataframe memory used after:  ", memory_after)

# Logistic Regression Model for Diabetes Prediction

Here we prepare data for machine learning, specifically using a logistic regression model to predict diabetes. First, numerical columns are identified and scaled using MinMaxScaler to ensure all features contribute equally to the model without bias from varying scales. A logistic regression model is then initialized with specific parameters. The dataset is split into features (X_log) and the target variable (y_log). The data is further divided into training and test sets to evaluate the model's performance on unseen data.

After training the logistic regression model, predictions are made on the test set. The model's effectiveness is assessed using accuracy, confusion matrix, and classification report, providing a comprehensive overview of its predictive capabilities in distinguishing between diabetic and non-diabetic individuals.

In [ ]:
# copy Dataframe for Logistic model
log_df = reduced_df.copy(deep=True)

# Selecting numerical columns (excluding binary/boolean columns)
numerical_columns = ["BMI", "GenHlth", "MentHlth", "PhysHlth", "Age"]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the numerical features
log_df[numerical_columns] = scaler.fit_transform(log_df[numerical_columns])

mylog_model = linear_model.LogisticRegression(solver="saga", max_iter=1000)

# 'X' is the feature set and 'y' is the target variable
X_log = log_df.drop("Diabetes", axis=1)
y_log = log_df["Diabetes"].astype("bool")  # Ensuring the target is boolean

# Splitting the dataset into the Training set and Test set
X_log_train, X_log_test, y_log_train, y_log_test = model_selection.train_test_split(
    X_log, y_log, test_size=0.25, random_state=42
)

# Train the model and output prediction of test data
mylog_model.fit(X_log_train, y_log_train)
y_pred_log = mylog_model.predict(X_log_test)

# Evaluate the model
accuracy_log = accuracy_score(y_log_test, y_pred_log)
conf_matrix_log = confusion_matrix(y_log_test, y_pred_log)
class_report_log = classification_report(y_log_test, y_pred_log)

print("\nLogistic Regression (single) prediction results:", "\n")
print(f"Accuracy: {round(accuracy_log*100,2)} %", "\n")
print("Confusion Matrix:")
print(conf_matrix_log, "\n")
print("Classification Report:")
print(class_report_log)

# Exporting the Model

To save the trained model, we can use the joblib library. Here we show how to save and load the model.

In [ ]:
import joblib

# Save the model to a file
joblib.dump(mylog_model, 'diabetes-classifier-log_model.pkl')

# Later, load the model from the file with the following:
# mylog_model = joblib.load('my_model_filename.pkl')

# Patient Outcome Prediction

This here is the objective of this project: to predict whether or not a patient is at risk of diabetes. The user enters the various health indicators, clicks the button and gets a prediction printed below.

### Age Category-number to Year-range

|*Category*|1|2|3|4|5|6|7|8|9|10|11|12|13|
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
|***Age range***|18-24|25-29|30-34|35-39|40-44|45-49|50-54|55-59|60-64|65-69|70-74|75-79|80+|

In [ ]:
# USER INTERFACE: form for input for patient prediction
import ipywidgets as widgets

# 'features_dict' is a dictionary mapping feature to description
features_dict = {
    "Sex": "Sex:",
    "Age": "Age category (1 = 18-24, 13 = 80 or older; see table):",
    "BMI": "Body Mass Index:",
    "HighBP": "High Blood Pressure",
    "HighChol": "High Cholesterol",
    "Smoker": "Have you smoked at least 100 cigarettes in your life?",
    "HvyAlcoholConsump": "Heavy drinkers (drinks <14 for men, <7 for women per week",
    "Stroke": "(Ever told) you had a Stroke?",
    "HeartDiseaseorAttack": "Heart Disease or Attack (CHD or MI)",
    "GenHlth": "General Health scale :",
    "MentHlth": "How many past days was your Mental Health not good?",
    "PhysHlth": "How many past days was your Physical Health not good?",
    "DiffWalk": "Do you have Difficulty Walking or climbing stairs?",
    "PhysActivity": "Physical Activity in past 30 days, not incl job",
    "Fruits": "Eat 1 Fruit or more per day",
    "Veggies": "Eat Veggies 1 or more per day",
}
widgets_dict = {}

# Create widgets for each feature
for item in features_dict.keys():
    if item in [
        "HighBP",
        "HighChol",
        "Smoker",
        "Stroke",
        "HeartDiseaseorAttack",
        "PhysActivity",
        "Fruits",
        "Veggies",
        "HvyAlcoholConsump",
        "DiffWalk",
    ]:
        # Binary features: create a dropdown with options 'Yes' and 'No'
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.RadioButtons(
            options={"No": 0, "Yes": 1},
            value=0,
        )
    if item in ["Sex"]:
        # Create a dropdown with options 'Male' and 'Female'
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.Dropdown(
            options=[("Female", 0), ("Male", 1)],
            value=0,
        )
    # Numerical features: create float sliders
    if item in ["BMI"]:
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.FloatSlider(
            value=20.0,
            min=10,
            max=50.0,
            step=0.1,
        )
    if item in ["GenHlth"]:
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.FloatSlider(
            value=3,
            min=1,
            max=5,
            step=1,
        )
    if item in ["MentHlth", "PhysHlth"]:
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.FloatSlider(
            value=0,
            min=0,
            max=30,
            step=1,
        )
    if item in ["Age"]:
        widgets_dict[item + "_label"] = widgets.Label(
            features_dict.get(item), layout={"width": "max-content"}
        )
        widgets_dict[item] = widgets.FloatSlider(
            value=8,
            min=1,
            max=13,
            step=1,
        )

# Button to make prediction
predict_btn = widgets.Button(description="Predict Patient Risk")

# Output widget to display prediction result
output = widgets.Output()


def on_predict_btn_clicked(b):
    # Prepare the input for the model
    input_data = [widgets_dict[feature].value for feature in features_dict.keys()]
    input_data = np.array(input_data).reshape(1, -1)

    # Create a DataFrame with input_data and assign column names using features
    input_df = pd.DataFrame(input_data, columns=features_dict.keys())

    # Ensure the DataFrame columns are in the correct order
    input_df = input_df[X_log.columns]

    # Apply the same scaling to the input as was done to the training data
    input_df[numerical_columns] = scaler.transform(input_df[numerical_columns])

    #### Make prediction ####
    prediction = mylog_model.predict(input_df)

    # Display prediction
    with output:
        output.clear_output()
        if prediction[0] == 0:
            print("Prediction: Not at risk of diabetes")
        else:
            print("Prediction: At risk of diabetes")


predict_btn.on_click(on_predict_btn_clicked)

# Display widgets
for widget in widgets_dict.values():
    display(widget)
display(predict_btn, output)

# Data Exploration

Next we generate some visuals to explore some possible relationships in the data. We will look at Histograms and Scatterplots.

## Histogram Figure 1.a

Here we generate a histogram that compares the distribution of general health ratings (GenHlth) between individuals with and without diabetes (Diabetes status) in the dataset. For each group, we display the frequency of respondents across five health rating categories, ranging from poor to excellent health, after these ratings have been reversed so that higher numbers indicate better health. The histogram for individuals without diabetes is shown in blue, while the histogram for those with diabetes is in orange, allowing for a visual comparison of how general health perceptions differ between these two groups. This visualization aids in understanding whether there's a noticeable difference in self-reported general health status among individuals based on their diabetes condition.

In [ ]:
# Filter the dataset by Diabetes status
gen_health_no_diabetes = reduced_df[reduced_df["Diabetes"] == False]["GenHlth"]
gen_health_with_diabetes = reduced_df[reduced_df["Diabetes"] == True]["GenHlth"]

# Plot histograms
plt.hist(
    gen_health_no_diabetes, bins=5, alpha=0.5, label="No Diabetes", edgecolor="black"
)
plt.hist(
    gen_health_with_diabetes, bins=5, alpha=0.5, label="Diabetes", edgecolor="black"
)

# Add legend
plt.legend()

# Add titles and labels as needed
plt.title("General Health by Diabetes Status")
plt.xlabel("General Health Rating")
plt.ylabel("Frequency")

# Set x-axis to display integer values from 1 to 5
plt.xticks(range(1, 6))

# Show the plot
plt.show()

## Histogram Figure 1.b

Analysis of physical health comparing diabetics to non-diabetics.

In [ ]:
# Filter the dataset by Diabetes status
phys_health_no_diabetes = reduced_df[reduced_df["Diabetes"] == False]["PhysHlth"]
phys_health_with_diabetes = reduced_df[reduced_df["Diabetes"] == True]["PhysHlth"]

# Plot histograms
plt.hist(
    phys_health_no_diabetes, alpha=0.5, label="No Diabetes", bins=6, edgecolor="black"
)
plt.hist(
    phys_health_with_diabetes, alpha=0.5, label="Diabetes", bins=6, edgecolor="black"
)

# Add legend
plt.legend()

# Add titles and labels as needed
plt.title("Physical Health by Diabetes Status")
plt.xlabel("Poor Physical Health Days")
plt.ylabel("Frequency")

# Show the plot
plt.show()

## Histogram Figure 1.c

Analysis of mental health comparing diabetics to non-diabetics.

In [ ]:
# Filter the dataset by Diabetes status
phys_health_no_diabetes = reduced_df[reduced_df["Diabetes"] == False]["MentHlth"]
phys_health_with_diabetes = reduced_df[reduced_df["Diabetes"] == True]["MentHlth"]

# Plot histograms
plt.hist(
    phys_health_no_diabetes, alpha=0.5, label="No Diabetes", bins=6, edgecolor="black"
)
plt.hist(
    phys_health_with_diabetes, alpha=0.5, label="Diabetes", bins=6, edgecolor="black"
)

# Add legend
plt.legend()

# Add titles and labels as needed
plt.title("Mental Health by Diabetes Status")
plt.xlabel("Poor Mental Health Days")
plt.ylabel("Frequency")

# Show the plot
plt.show()

## Histogram Figure 1.d

### Age distribution

Here we provide a visual distribution analysis of the Age feature. Of note, the data was collected using a 13-level age category where 1 = 18-24, 9 = 60-64, and 13 = 80 or older.

In [ ]:
# Plot Age histogram
graph_histograms = reduced_df.hist(column="Age", grid=True, bins=13)

# Calculate the tick positions for 13 bins
tick_positions = range(1, 14)

# Set the x-axis ticks
plt.xticks(tick_positions)

# Optionally, set x-axis and y-axis labels
plt.xlabel("Age Category")
plt.ylabel("Frequency")

# Show the plot
plt.show()

## Scatterplot figure 2.a

### BMI over Age

Here we see that diabetics have a higher BMI on average.

In [ ]:
# Copy the DataFrame to avoid modifying the original data
plot_df = reduced_df.copy()

# Map True/False to Male/Female
plot_df["Sex"] = plot_df["Sex"].map({True: "Male", False: "Female"})

# Age/BMI Scatterplot
sns.lmplot(
    data=plot_df,
    x="Age",
    y="BMI",
    col="Sex",
    hue="Diabetes",
    x_bins=1000,
    fit_reg=False,
)

## Scatterplot figure 2.b

### General Health over Age

The general health indicator is on a scale of 1 to 5, with a score of 5 as excellent. Here we see that diabetics have poorer general health. We also see a trend in healthy people where the older one gets, the poorer their overall health. Diabetics do not show this trend.

In [ ]:
# Age/General-Health Scatterplot
sns.lmplot(data=reduced_df, x="Age", y="GenHlth", hue="Diabetes", x_bins=1000)

## Scatterplot figure 2.c

### Physical Health as compared to Mental health

Here we see a trend that the more days per month of poor mental health an individual has, the more days on average of physical health they will experience. Of note is diabetics have a similar distribution of poor mental health days, but significantly more days per month of poor physical health.

In [ ]:
# Health Scatterplot
sns.lmplot(data=reduced_df, x="MentHlth", y="PhysHlth", hue="Diabetes", x_bins=1000)

# Random Forest Model for Comparison

Here we set up a second model using the Random Forest algorithm to compare with our Logistic Regression model. The steps are much the same as before, however we do not need to scale the numerical data.

In [ ]:
# 'X' is the set of features and 'y' is the target variable
X_rf = reduced_df.drop("Diabetes", axis=1)
y_rf = reduced_df["Diabetes"].astype("bool")  # Ensuring the target is boolean

# Splitting the dataset into the Training set and Test set
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(
    X_rf, y_rf, test_size=0.25, random_state=42
)

# Creating a Random Forest Classifier -- You can adjust parameters
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fitting Random Forest to the Training set
rf_classifier.fit(X_rf_train, y_rf_train)

# Predicting the Test set results
y_pred_rf = rf_classifier.predict(X_rf_test)

accuracy_rf = accuracy_score(y_rf_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_rf_test, y_pred_rf)
class_report_rf = classification_report(y_rf_test, y_pred_rf)

# Evaluate the model
print("\nRandom Forest (single) prediction results:", "\n")
print(f"Accuracy: {round(accuracy_rf*100,2)} %", "\n")
print("Confusion Matrix:")
print(conf_matrix_rf, "\n")
print("Classification Report:")
print(class_report_rf)

# Cross-Validation of Models

Next we perform a cross-validation process on the logistic regression and Random Forest models to verify their reliability across different subsets of the dataset. By utilizing the KFold method with 5 splits and shuffling enabled, the dataset is divided into distinct subsets to conduct multiple training and testing cycles. The average of these scores is calculated and displayed, offering a robust measure of the model's overall performance. This approach helps to ensure that the model's predictive accuracy is not overly dependent on any particular partition of the data, thereby increasing confidence in its generalizability.

In [ ]:
# Verify model by averaging different test/train splits
k_folds = KFold(n_splits=5, shuffle=True)
# The number of folds determines the test/train split for each iteration.
# So 5 folds has 5 different mutually exclusive training sets.
# That's a 1 to 4 (or .20 to .80) testing/training split for each of the 5 iterations.

# This is the average score. Print 'scores' to see array of individual iteration scores.
log_scores = cross_val_score(mylog_model, X_log, y_log)
rf_scores = cross_val_score(rf_classifier, X_rf, y_rf)

# Output average scores
print(
    "Logistic Regression Average Prediction Score: ",
    round(log_scores.mean() * 100, 2),
    "%",
)
print("Random Forest Average Prediction Score: ", round(rf_scores.mean() * 100, 2), "%")

# Confusion Matrix Visual

Confusion matrixes are used to evaluate the performance in classifying diabetic and non-diabetic individuals. Here we look at the Logistic Regression model, because it proved to be more accurate. More importantly, it had less false-negative outcomes compared to the random forest model (4101 vs 4297).

In [ ]:
# Plot confusion matrix
graph_confusion_matrix = ConfusionMatrixDisplay.from_predictions(y_log_test, y_pred_log)